In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [ ]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


: 

In [ ]:
class Model_LinearSumOfElementCombinationWithPower_ForMultipleRegression(
    ModelBaseForMultipleRegression
):
    """説明変数の組み合わせたうえで片方をn乗にするモデル

    Model_LinearSumOfElementCombinationWithPower_ForMultipleRegression

    Attributes:
        explanatoryVariableColumnNames (list[str]): 説明変数の列名のリスト
        rawExplanatoryVariable (pd.DataFrame): 説明変数のデータフレーム
        rawExplanatoryVariableForTest (pd.DataFrame): テスト用の説明変数のデータフレーム。説明変数のデータフレームと同様の値が入っている(?)
        rawResponseVariable (pd.DataFrame): 目的変数のデータフレーム
        rawResponseVariableForTest (pd.DataFrame): テスト用の目的変数のデータフレーム。目的変数のデータフレームと同様の値が入っている(?)
        responseVariableColumnNames (list[str]): 目的変数の列名のリスト
    """

    def __init__(
        self,
        inputDF,
        explanatoryVariableColumnNames: list[str],
        exponent: int,
        responseVariableColumnNames: list[str],
        conditionDictForTest: Dict[str, str] = {},
        targetDF: pd.DataFrame = None,
    ):
        super().__init__(
            inputDF=inputDF,
            explanatoryVariableColumnNames=explanatoryVariableColumnNames,
            responseVariableColumnNames=responseVariableColumnNames,
            conditionDictForTest=conditionDictForTest,
            targetDF=targetDF,
        )
        self.exponent: int = exponent

    def build_model(self) -> bool:
        """build_model(self) -> bool

        オブジェクトの初期化時に生成された、インスタンスの説明変数及びインスタンスの目的変数からモデルを構築する

        Args:
            self :none

        Returns: boolean。成功ならTrue,失敗ならFalse
        """

        df_mid_var: pd.DataFrame = self.return_df_for_combinations(
            inputDF=self.rawExplanaoryVariable, exponent=self.exponent
        )

        self.lr = LinearRegression()
        self.lr.fit(df_mid_var, self.rawResponseVariable)

        return True

    def predict(self, inputDF: pd.DataFrame) -> np.ndarray:
        """predict(self, inputDF: pd.DataFrame) -> np.ndarray

        Args:
            self: none
            inputDF (pd.DataFrame) :構築されたモデルを使って予測を行うDF

        Returns:
            np.ndarray
        """

        df_mid_var: pd.DataFrame = self.return_df_for_combinations(
            inputDF, exponent=self.exponent
        )
        resultDF = self.lr.predict(df_mid_var)

        return resultDF

    def returnMAPE(self) -> float:
        """returnMAPE() -> float

        モデルに構築に使用したデータからMAPEを算出する。

        Args:
            self: none

        Returns:
            float: 「モデルの構築に用いたデータから予測された値」と「実際の値」から算出されたMAPE
            int: 失敗した場合,-1

        """

        return_expect: np.ndarray = self.rawResponseVariable[
            self.responseVariableColumnNames
        ].values
        return_actually: np.ndarray = self.predict(self.rawExplanaoryVariable)

        return returnMapeScore(return_expect, return_actually)

    def return_df_for_combinations(
        self, inputDF: pd.DataFrame, exponent: int
    ) -> pd.DataFrame:
        """return_df_for_combinations(self, inputDF: pd.DataFrame) -> pd.DataFrame

        入力DFから説明変数の組み合わせを算出し、その組み合わせの要素同士を累乗計算をしつつかけ合わせた列で構成されたDFを返す関数

        Args:
            inputDF (pd.DataFrame):入力DF
            exponent (int): 累乗計算時に用いる指数
        """

        returnDF: pd.DataFrame = inputDF.copy(deep=True)
        returnDF_columns = returnDF.columns.tolist()
        returnDF = returnDF.drop(returnDF_columns, axis=1)

        list_combinations: list[set[str]] = list(
            itertools.combinations(self.explanatoryVariableColumnNames, 2)
        )
        for combination_index in range(len(list_combinations)):
            combination: set[str, str] = list_combinations[combination_index]
            exp_name0: str = combination[0]
            exp_name1: str = combination[1]
            returnDF["col" + str(combination_index)] = (
                inputDF[exp_name0] ** exponent * inputDF[exp_name1]
            )
        for combination_index in range(len(list_combinations)):
            combination: set[str, str] = list_combinations[combination_index]
            exp_name0: str = combination[0]
            exp_name1: str = combination[1]
            returnDF["col" + str(combination_index + len(list_combinations))] = (
                inputDF[exp_name0] * inputDF[exp_name1] ** exponent
            )

        return returnDF


def test_Model_LinearSumOfElementCombinationWithPower_ForMultipleRegression():
    """test_Model_LinearSumOfElementCombinationWithPower_ForMultipleRegression()

    クラス Model_LinearSumOfElementCombinationWithPower_ForMultipleRegression のテスト

    """

    # 説明変数
    plotX_1: np.ndarray = np.linspace(10, 20, 11)
    plotX_2: np.ndarray = 13 * np.linspace(20, 10, 11)
    plotX_3: np.ndarray = 23 * np.linspace(10, 20, 11)
    plotX_4: np.ndarray = 29 * np.linspace(30, 40, 11)

    # 係数・切片
    a: int = 17
    b: int = 19
    c: int = 23
    d: int = 29
    e: int = 17
    f: int = 19
    z: int = 31

    # 目的変数
    plotT: np.ndarray = (
        (a * plotX_2)
    )

    # DFの作成
    columnNames: list[str] = ["process", "plotX_2", "plotX_3", "plotT"]
    datumForDF: list[np.ndarray] = [plotX_1, plotX_2, plotX_3, plotT]
    inputDFForTest: pd.DataFrame = pd.DataFrame(index=columnNames, data=datumForDF).T
    inputDFForTest["functionName"] = "functionName"

    # 説明変数のカラム名のリスト
    columnNamesForExp: list[str] = ["process", "plotX_2", "plotX_3"]
    # 目的変数のカラム名のリスト
    columnNamesForRes: list[str] = ["plotT"]

    # 予測の実施
    objectModel = Model_LinearSumOfElementCombinationWithPower_ForMultipleRegression(
        inputDF=inputDFForTest,
        explanatoryVariableColumnNames=columnNamesForExp,
        responseVariableColumnNames=columnNamesForRes,
        conditionDictForTest={},
        exponent=2,
    )

    # モデルの構築時に使用されるメソッドのテスト
    col0: np.ndarray = plotX_1 * plotX_1 * plotX_2
    col1: np.ndarray = plotX_1 * plotX_1 * plotX_3
    col2: np.ndarray = plotX_2 * plotX_2 * plotX_3
    col3: np.ndarray = plotX_1 * plotX_2 * plotX_2
    col4: np.ndarray = plotX_1 * plotX_3 * plotX_3
    col5: np.ndarray = plotX_2 * plotX_3 * plotX_3
    inputDictForExpectedDF: dict[str, np.ndarray] = {
        "col0": col0,
        "col1": col1,
        "col2": col2,
        "col3": col3,
        "col4": col4,
        "col5": col5,
    }
    expectedDF: pd.DataFrame = pd.DataFrame(data=inputDictForExpectedDF)
    actuallyDF: pd.DataFrame = objectModel.return_df_for_combinations(inputDFForTest, 2)

    assert actuallyDF.equals(
        expectedDF
    ), f"actuallyDF.head()=\n{actuallyDF.head()}\nexpectedDF.head()=\n{expectedDF.head()}"

    # モデルの構築
    objectModel.build_model()
    # モデル構築に用いたデータと予測されたデータとのMAPEを比較して、実装ができているかを確認する
    mape: float = objectModel.returnMAPE()
    assert 0 <= mape < 1, f"mape = {mape}"

: 

***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___